# Búsqueda hiperparámteros del modelo RAF 



- Cada bloque de 1000 días tiene su propia dinámica, volatilidad y características del mercado. Los hiperparámetros óptimos pueden variar de bloque a bloque.

- Sin embargo si pretendemos calcular hiperparámetros de cada bloque y de cada acción, los tiempos de cómputo se disparan.

- Por ello realizaremos búsqueda de hiperparámetros usando un bloque y una acción determinados. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

# Escalamiento
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Métricas
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error, r2_score

# modelo RAF
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit

# hiperparametros
from sklearn.model_selection import GridSearchCV


## Lectura de datasets 

In [2]:
# Nombre de las acciones
tickers = [
    'TSCO', 'AZN', 'BARC', 'BP', 'BATS', 'HLMA',
    'HSBA', 'JMAT', 'LGEN', 'MKS', 'PSON', 'REL',
    'NWG', 'SHEL', 'SGE', 'SBRY', 'SDR', 'SVT',
    'SMIN', 'SSE', 'VOD'
]

data_frames = {} # Inicializo diciconario de data frames

for ticker in tickers: 
    
    df = pd.read_csv(f'datasets_features/{ticker}.csv')  # leo el dataset
    data_frames[ticker] = df                             # guardo dataframes
    
print(data_frames.keys())

dict_keys(['TSCO', 'AZN', 'BARC', 'BP', 'BATS', 'HLMA', 'HSBA', 'JMAT', 'LGEN', 'MKS', 'PSON', 'REL', 'NWG', 'SHEL', 'SGE', 'SBRY', 'SDR', 'SVT', 'SMIN', 'SSE', 'VOD'])


## Función que calcula hiperparametros (1 acción - 1 bloque)

In [3]:
def hiperparametros_RAF(df_bloque):
    '''
    Dado un bloque de dataframe (1000 días, o todo el dataset) calcula los hiperámetros óptimos 
    ''' 
    # 1) ------------- Partición de datos ----------------
    
    # Índice de separación
    N = len(df_bloque)
    split_idx = int(N * 0.75) 
    
    df_features = df_bloque.drop(columns=['Date', 'Open', 'High', 'Low','Close','Volume', 'prev_close', 'target']) # todas columnas r_i
    target      = df_bloque['target']  # columna del target

    
    X_train = df_features.iloc[ : split_idx]
    X_test  = df_features.iloc[split_idx : ]
    y_train = target.iloc[ : split_idx]
    y_test  = target.iloc[split_idx : ]
    
    #print(split_idx)
    #display(X_train.head())
    display(X_train.shape)
    display(X_test.shape)

    # 2) ------------ Estandarización ---------------------
    # Features 
    scaler_x = MinMaxScaler(feature_range=(-1,1))
    X_train_s = scaler_x.fit_transform(X_train)     
    X_test_s  = scaler_x.transform(X_test)

    # Target (al estandarizar son 2D)
    scaler_y = MinMaxScaler(feature_range=(-1,1))
    y_train_s = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()    # Necesario reshape, scaler espera 2D      
    y_test_s  = scaler_y.transform(y_test.values.reshape(-1, 1))


    # 3) ------------- Busco hiperparametros --------------------
    # Grid 
    param_grid = {'n_estimators': [100, 150],
                  'max_features': [1.0, 2, 3, 5, 6, 7, 8],   #default 1.0, None o 1. = max_features=n_features
                  'max_depth'   : [None, 10, 20, 25, 30]     #default=None, profundidad del arbol hasta que hay hojas puras 
                 }

    # Búsqueda por grid search con validación cruzada
    grid = GridSearchCV(
                estimator  = RandomForestRegressor(random_state = 123,
                                                   criterion    = 'squared_error', #por defecto sq_error
                                                   oob_score    = False,
                                                   n_jobs       = -1
                                                  ),
                param_grid = param_grid,
                # 'neg_mean_squared_error' devuelve el negativo del MSE → cuanto más alto (menos negativo), mejor (menos error)
                #scoring    = 'neg_mean_squared_error', 
                scoring    = None, #'neg_root_mean_squared_error', # None= el que se use en estimator
                n_jobs     = - 1,
                cv         = TimeSeriesSplit(n_splits=5), 
                refit      = True, # por defecto True
                verbose    = 0,
                return_train_score = True #defecto false
           )

    grid.fit(X=X_train, y=y_train)
    
    #medias_test     = grid.cv_results_['mean_test_score']  #array
    #std_test        = grid.cv_results_['std_test_score']   #array
    #posibles_params = grid.cv_results_['params']           #lista de diccionarios

    # Almaceno resultado
    resultado = grid.best_params_.copy()
    resultado['best_score'] = grid.best_score_  #añado una clave más
    
    return resultado
    
    

### Ejemplo búsqueda hiperparámetros en un bloque  

In [4]:
# Elijo un data set
df = data_frames['AZN']

# Calculo hiperparámetros (1 bloque)
hiper_1bloque = hiperparametros_RAF(df[0:1000])

(750, 20)

(250, 20)

In [5]:
# Calculo hiperparámetros (todos los bloque)
hiper = hiperparametros_RAF(df)

(4817, 20)

(1606, 20)

In [6]:
display(hiper_1bloque)
display(hiper)

{'max_depth': 10,
 'max_features': 2,
 'n_estimators': 150,
 'best_score': -0.06797944897575321}

{'max_depth': 10,
 'max_features': 2,
 'n_estimators': 100,
 'best_score': -0.01550373783452943}

## Nota: 

Vemos que el resultado es muy similar, sin embargo el tiempo de cómputo es mucho mayor si usamos todos los bloques temporales de golpe. Usaremos por tanto, un solo bloque para búsqueda de hiperparámteros. 

**Nota:** 

En un principio se intentó calcular los hiperparámetros en cada uno de los bloques. Sin embargo, el tiempo de cómputo se incrementó. Los resultados se muestran en el dataframe inferior, y se vió que los hiperparámetros no varía demasiado entre bloques. 

Finalmente se optó por usar todos los bloques y elegir una acción representativa (AZN). 

In [80]:
# Este es el resultado calculado en su momento (No ejecutar celda)
df_resultado = pd.DataFrame.from_dict(resultado_hiperparametros, orient='index').reset_index()
df_resultado
df_resultado.sort_values(by='best_score', ascending=True)

,index,max_depth,max_features,n_estimators,best_score
11,Bloque_12,20,2,150,-0.155027
15,Bloque_16,10,2,150,-0.101347
9,Bloque_10,10,2,150,-0.097732
10,Bloque_11,10,2,150,-0.076573
6,Bloque_7,10,2,100,-0.075156
0,Bloque_1,10,2,150,-0.067979
17,Bloque_18,10,2,150,-0.062841
14,Bloque_15,10,2,150,-0.058557
19,Bloque_20,10,2,150,-0.056747
12,Bloque_13,30,2,150,-0.054821


In [86]:
resultado

{'max_depth': 10,
 'max_features': 2,
 'n_estimators': 100,
 'best_score': -0.01550373783452943}